In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Huggingface install & import
https://github.com/huggingface/neuralcoref 
https://stackoverflow.com/questions/57008612/kernel-died-when-running-neuralcoref 

In [ ]:
pip install -U spacy==2.1.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 38.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install Cython --install-option="--no-cython-compile"

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall -y neuralcoref 

!pip install neuralcoref --no-binary neuralcoref

Found existing installation: neuralcoref 4.0
Uninstalling neuralcoref-4.0:
  Successfully uninstalled neuralcoref-4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached neuralcoref-4.0.tar.gz (368 kB)
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
    Running setup.py install for neuralcoref ... done


In [ ]:
import neuralcoref
import spacy
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

100%|██████████| 40155833/40155833 [00:04<00:00, 8219330.56B/s] 


# Setup

In [ ]:
data_path = "/content/drive/My Drive/AmbiCoref/data_ealc/"

In [ ]:
import pandas as pd
col_names = ['Category','%Name1','%Name2','%Missing','%Both','%Other','Model']
df_all = pd.DataFrame(columns = col_names)

# Type ECO

## unambiguous ECO-1 active

In [ ]:
file = open(data_path+"sentences/ECO-1_unambiguous.txt", "r")
sentences = file.readlines()

In [ ]:
len(sentences)

11336

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[span_index_that+1:span_index_that+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECO-1_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## ambiguous ECO-1 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECO-1_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

11336

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[span_index_that+1:span_index_that+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECO-1_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## unambiguous ECO-2 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECO-2_unambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

11336

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[span_index_that+1:span_index_that+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECO-2_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## ambiguous ECO-2 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECO-2_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

11336

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[span_index_that+1:span_index_that+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECO-2_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])



# Type ECS

## unambiguous ECS-1 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECS-1_unambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

4472

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[-3:-2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECS-1_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## ambiguous ECS-1 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECS-1_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

4472

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[-3:-2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total


In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECS-1_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## unambiguous ECS-2 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECS-2_unambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

4472

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[-3:-2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total


In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECS-2_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])


## ambiguous ECS-2 active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/ECS-2_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

4472

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_that = s.index('that')
  word_index_that = words.index("that")
  span_index_that = spans.index("that")
  subString_before_that = s[:ch_index_that]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_that:
    name2 = doc[span_index_that-1:span_index_that]
    num_span_name2 = 2 + words[word_index_that-1].count("-")*2
    name2 = doc[span_index_that-num_span_name2:span_index_that]
  else:
    name2 = doc[span_index_that-1:span_index_that]

  pronoun = doc[-3:-2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "ECS-2_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])

# Type IC

## unambiguous active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/IC_unambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

8424

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_bc = s.index('because')
  word_index_bc = words.index("because")
  span_index_bc = spans.index("because")
  subString_before_bc = s[:ch_index_bc]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_bc:
    name2 = doc[span_index_bc-1:span_index_bc]
    num_span_name2 = 2 + words[word_index_bc-1].count("-")*2
    name2 = doc[span_index_bc-num_span_name2:span_index_bc]
  else:
    name2 = doc[span_index_bc-1:span_index_bc]

  pronoun = doc[span_index_bc+1:span_index_bc+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "IC_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])

## ambiguous active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/IC_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

8424

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1

  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')



  ch_index_bc = s.index('because')
  word_index_bc = words.index("because")
  span_index_bc = spans.index("because")
  subString_before_bc = s[:ch_index_bc]

  name1 = doc[0:1]
  name2 = None
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 

  if " the " in subString_before_bc:
    name2 = doc[span_index_bc-1:span_index_bc]
    num_span_name2 = 2 + words[word_index_bc-1].count("-")*2
    name2 = doc[span_index_bc-num_span_name2:span_index_bc]
  else:
    name2 = doc[span_index_bc-1:span_index_bc]

  pronoun = doc[span_index_bc+1:span_index_bc+2]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total


In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "IC_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])

# Type TOP

## unambiguous active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/TOP_unambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

8424

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1


  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')

  name1 = doc[0:1]
  name2_index_start = 0
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 
    name2_index_start = num_span_name1 + 1
  
  if "by" in words:
    span_index_by = spans.index("by")
    name2_index_start = span_index_by + 1

  if spans[name2_index_start] == 'the':
    name2 = doc[name2_index_start:name2_index_start+2]
  else:
    name2 = doc[name2_index_start:name2_index_start+1]

  span_index_pronoun = -1
  if "she" in words:
    span_index_pronoun = spans.index("she")
  else:
    span_index_pronoun = spans.index("he")
  pronoun = doc[span_index_pronoun:span_index_pronoun+1]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1
  
  

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total


In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "TOP_unambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])

## ambiguous active

In [ ]:
file = open(r"/content/drive/My Drive/AmbiCoref/data_ealc/sentences/TOP_ambiguous.txt","r")
sentences = file.readlines()

In [ ]:
len(sentences)

8424

In [ ]:
count_pos1 = 0
count_pos2 = 0
count_total = 0
count_other = 0
count_both = 0


for s in sentences:
  count_total += 1


  doc = nlp(s)
  words = s.split(" ")
  spans = s.replace('-', ' - ').split(' ')

  name1 = doc[0:1]
  name2_index_start = 0
  if words[0] == 'The': 
    num_span_name1 = 2 + words[1].count("-")*2
    name1 = doc[:num_span_name1] 
    name2_index_start = num_span_name1 + 1
  
  if "by" in words:
    span_index_by = spans.index("by")
    name2_index_start = span_index_by + 1

  if spans[name2_index_start] == 'the':
    name2 = doc[name2_index_start:name2_index_start+2]
  else:
    name2 = doc[name2_index_start:name2_index_start+1]

  span_index_pronoun = -1
  if "she" in words:
    span_index_pronoun = spans.index("she")
  else:
    span_index_pronoun = spans.index("he")
  pronoun = doc[span_index_pronoun:span_index_pronoun+1]

  if not pronoun._.is_coref:
    continue
  cluster = pronoun._.coref_cluster

  if cluster is None:
    continue

  if name1 in cluster:
    if name2 not in cluster:
      count_pos1 += 1
    else:
      count_both += 1
  elif name2 in cluster:
    count_pos2 += 1
  elif len(cluster) > 1 :
    count_other += 1
  
  

#### Results

In [ ]:
name1_perc = count_pos1 / count_total
name2_perc = count_pos2 / count_total
missing_perc = (count_total -count_pos1 - count_pos2 - count_other - count_both) / count_total
both_perc = count_both / count_total
other_perc = count_other / count_total

In [ ]:
result_temp = [(name1_perc, name2_perc, missing_perc, both_perc, other_perc)]
df_temp = pd.DataFrame(result_temp, columns=['%Name1','%Name2','%Missing','%Both','%Other'] )
df_temp['Category'] = "TOP_ambiguous_active" 
df_temp['Model'] = "Hugging_face" 
df_all = pd.concat([df_all, df_temp])

# Final output

In [ ]:
df_all.to_csv(data_path+"results_Hugging_face.csv")